In [1]:
import numpy as np 
import pandas as pd
import os
import h5py
from PIL import Image
from torchinfo import summary
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
train_link = "/kaggle/input/fer2013/train"
test_link = "/kaggle/input/fer2013/test"

In [3]:
def label_encoder(text):
    dict = {"angry":0,"disgust":1,"fear":2,"happy":3,"neutral":4,"sad":5,"surprise":6}
    for expression, label in dict.items():
        if expression in text.lower():
            return label
    return None
    

In [4]:
def dataset_creator(link_of_files, save_dir_category: str):
    features = []
    labels = []
    target_size = (48,48)
   
    
    for dirname, _, filenames in os.walk(link_of_files):
        for filename in filenames:
            
            image_path = os.path.join(dirname, filename)
            img = np.asarray(Image.open(image_path).resize(target_size, Image.Resampling.LANCZOS), dtype = np.float32)
            features.append(img)
            
            label = label_encoder(image_path)
            labels.append(label)
            
    features = np.array(features)
    labels = np.array(labels)

    #shuffle the data
    
    indices = np.arange(features.shape[0])  
    np.random.shuffle(indices)          
    shuffled_data = features[indices]
    shuffled_labels = labels[indices]

    # save the dataset in .h5 format
    
    saving_path = f"{save_dir_category}.h5"
    with h5py.File(saving_path, "w") as h5_file:
        h5_file.create_dataset("features", data = shuffled_data)
        h5_file.create_dataset("labels", data = shuffled_labels)
    print("success")

    return features, labels
    

In [5]:
train_features, train_labels = dataset_creator(train_link, "train_data")
test_features, test_labels = dataset_creator(test_link, "test_data")

success
success


In [6]:
train_features.shape, train_labels.shape, train_labels.dtype, train_features.dtype

((28709, 48, 48), (28709,), dtype('int64'), dtype('float32'))

In [7]:
test_features.shape, test_labels.shape, test_labels.dtype, test_features.dtype

((7178, 48, 48), (7178,), dtype('int64'), dtype('float32'))

In [8]:
def data_reader(h5_file_link):
    with h5py.File(h5_file_link, 'r') as h5_file:
        features = h5_file['features'][:]
        labels = h5_file['labels'][:]

    return features, labels

In [9]:
test_features, test_labels = data_reader("/kaggle/working/test_data.h5")
train_features, train_labels = data_reader("/kaggle/working/train_data.h5")

In [10]:
test_features.shape, train_features.shape

((7178, 48, 48), (28709, 48, 48))

In [11]:
from torch.utils.data import Dataset, DataLoader
import torch

In [12]:
X_train_tensors = torch.tensor(train_features.astype(np.float32)/255)
y_train_tensors = torch.tensor(train_labels.astype(np.float32))

In [13]:
X_test_tensors = torch.tensor(test_features.astype(np.float32)/255)
y_test_tensors = torch.tensor(test_labels.astype(np.float32))

In [14]:
X_test_tensors.dtype, y_test_tensors.dtype

(torch.float32, torch.float32)

In [15]:
X_test_tensors.shape, y_test_tensors.shape

(torch.Size([7178, 48, 48]), torch.Size([7178]))

In [16]:
X_train_tensors.dtype, y_train_tensors.dtype

(torch.float32, torch.float32)

In [17]:
X_train_tensors.shape, y_train_tensors.shape

(torch.Size([28709, 48, 48]), torch.Size([28709]))

In [18]:
X_train_tensors.unsqueeze_(1).shape
y_train_tensors = (y_train_tensors).long()

In [19]:
X_test_tensors.unsqueeze_(1).shape
y_test_tensors = (y_test_tensors).long()

In [20]:
print(torch.isnan(X_train_tensors).sum())  # Check for NaN in features
print(torch.isinf(y_train_tensors).sum()) # Check for Inf in features


tensor(0)
tensor(0)


In [21]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [22]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()

        # First convolutional block
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.2)
        )

        # Second convolutional block
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout2d(0.2)
        )

        # Third convolutional block
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout2d(0.2)
        )

        # Pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((6, 6))

        # Fully connected layers
        self.fc1 = nn.Linear(256 * 6 * 6, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, num_classes)

        # Dropout for fully connected layers
        self.dropout = nn.Dropout(0.5)

        # Activation
        self.relu = nn.ReLU()

    def forward(self, x):
        # First block
        x = self.conv1(x)
        x = self.pool(x)

        # Second block
        x = self.conv2(x)
        x = self.pool(x)

        # Third block
        x = self.conv3(x)
        x = self.adaptive_pool(x)

        # Flatten
        x = x.view(x.size(0), -1)

        # Fully connected layers
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

    @staticmethod
    def train_model():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.train()

        # Track metrics
        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(epochs):
            running_loss = 0
            correct = 0
            total = 0

            for batch_idx, (features, labels) in enumerate(train_dataloader):
                features, labels = features.to(device), labels.to(device)

                # Forward pass
                outputs = model(features)
                loss = criterion(outputs, labels)

                # Backward pass
                optimizer.zero_grad()
                loss.backward()

                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                # Update weights
                optimizer.step()

                # Calculate accuracy
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                running_loss += loss.item()

                # Print batch progress
                # if (batch_idx + 1) % 50 == 0:
                #     print(f'Epoch [{epoch+1}/{epochs}], '
                #           f'Step [{batch_idx+1}/{len(train_dataloader)}], '
                #           f'Loss: {loss.item():.4f}, '
                #           f'Acc: {100 * correct/total:.2f}%')

            # Calculate epoch metrics
            epoch_loss = running_loss / len(train_dataloader)
            epoch_acc = 100 * correct / total

            print(f'Epoch {epoch+1}: Loss={epoch_loss:.4f}, Accuracy={epoch_acc:.2f}%')

            # Early stopping check
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 5:  # 5 epochs patience
                    print("Early stopping triggered")
                    break

    @staticmethod
    def model_eval():
        model.eval()
        correct_pred = 0
        total_pred = 0

        # For confusion matrix
        all_preds = []
        all_labels = []

        with torch.no_grad():
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            for features, labels in test_dataloader:
                features, labels = features.to(device), labels.to(device)

                # Forward pass
                outputs = model(features)
                _, predicted = torch.max(outputs, 1)

                # Store predictions and labels
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                # Calculate accuracy
                correct_pred += (predicted == labels).sum().item()
                total_pred += labels.size(0)

            # Calculate metrics
            accuracy = 100 * correct_pred / total_pred

            # Per-class accuracy
            class_correct = [0] * 7
            class_total = [0] * 7

            for pred, label in zip(all_preds, all_labels):
                if pred == label:
                    class_correct[label] += 1
                class_total[label] += 1

            # Print results
            print(f'\nOverall Accuracy: {accuracy:.2f}%')
            for i in range(7):
                if class_total[i] > 0:
                    class_acc = 100 * class_correct[i] / class_total[i]
                    print(f'Accuracy of class {i}: {class_acc:.2f}%')

            return accuracy

In [23]:
train_data = CustomDataset(X_train_tensors, y_train_tensors)
test_data = CustomDataset(X_test_tensors,y_test_tensors)

In [24]:
train_dataloader = DataLoader(train_data, batch_size = 256, shuffle = True)
test_dataloader = DataLoader(test_data, batch_size = 64, shuffle = False)

In [25]:
len(test_dataloader), len(train_dataloader)

(113, 113)

In [26]:
model = NeuralNetwork(num_classes=7).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
epochs=100

In [27]:
summary(model, input_size = (256,1, 48, 48))

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [256, 7]                  --
├─Sequential: 1-1                        [256, 64, 48, 48]         --
│    └─Conv2d: 2-1                       [256, 64, 48, 48]         640
│    └─BatchNorm2d: 2-2                  [256, 64, 48, 48]         128
│    └─ReLU: 2-3                         [256, 64, 48, 48]         --
│    └─Dropout2d: 2-4                    [256, 64, 48, 48]         --
├─MaxPool2d: 1-2                         [256, 64, 24, 24]         --
├─Sequential: 1-3                        [256, 128, 24, 24]        --
│    └─Conv2d: 2-5                       [256, 128, 24, 24]        73,856
│    └─BatchNorm2d: 2-6                  [256, 128, 24, 24]        256
│    └─ReLU: 2-7                         [256, 128, 24, 24]        --
│    └─Dropout2d: 2-8                    [256, 128, 24, 24]        --
├─MaxPool2d: 1-4                         [256, 128, 12, 12]        --
├─Sequen

In [28]:
model.train_model()

Epoch 1: Loss=1.7804, Accuracy=28.53%
Epoch 2: Loss=1.5575, Accuracy=39.57%
Epoch 3: Loss=1.4668, Accuracy=43.22%
Epoch 4: Loss=1.4059, Accuracy=45.87%
Epoch 5: Loss=1.3631, Accuracy=47.91%
Epoch 6: Loss=1.3299, Accuracy=49.29%
Epoch 7: Loss=1.2987, Accuracy=50.17%
Epoch 8: Loss=1.2711, Accuracy=51.46%
Epoch 9: Loss=1.2413, Accuracy=52.52%
Epoch 10: Loss=1.2235, Accuracy=53.14%
Epoch 11: Loss=1.1995, Accuracy=54.25%
Epoch 12: Loss=1.1806, Accuracy=54.89%
Epoch 13: Loss=1.1577, Accuracy=56.28%
Epoch 14: Loss=1.1343, Accuracy=56.96%
Epoch 15: Loss=1.1089, Accuracy=57.73%
Epoch 16: Loss=1.0931, Accuracy=58.54%
Epoch 17: Loss=1.0700, Accuracy=59.41%
Epoch 18: Loss=1.0523, Accuracy=60.08%
Epoch 19: Loss=1.0300, Accuracy=60.74%
Epoch 20: Loss=1.0091, Accuracy=61.50%
Epoch 21: Loss=0.9929, Accuracy=62.14%
Epoch 22: Loss=0.9704, Accuracy=63.22%
Epoch 23: Loss=0.9546, Accuracy=63.43%
Epoch 24: Loss=0.9279, Accuracy=64.61%
Epoch 25: Loss=0.9115, Accuracy=65.47%
Epoch 26: Loss=0.8881, Accuracy=66

In [29]:
accuracy = model.model_eval()


Overall Accuracy: 62.27%
Accuracy of class 0: 52.61%
Accuracy of class 1: 56.76%
Accuracy of class 2: 44.92%
Accuracy of class 3: 82.69%
Accuracy of class 4: 57.02%
Accuracy of class 5: 51.16%
Accuracy of class 6: 76.41%


In [30]:
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 62.27%


In [31]:
import torch
import json
import os

model_dir = "."
torch.save(model.state_dict(), f"{model_dir}/pytorch_model.bin")

label_map = {"angry":0, "disgust":1, "fear":2, "happy":3, "neutral":4, "sad":5, "surprise":6}
with open(f"{model_dir}/label_map.json", "w") as f:
    json.dump(label_map, f)

In [32]:
!pip install huggingface_hub
# !huggingface-cli login --token <ENTER YOUR HF TOKEN HERE>

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `finwrite` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `finwrite`


In [33]:
from huggingface_hub import login, upload_file

repo_id = "Akshit-77/fer-emotions-model"

def update_model_on_hub(repo_id, model_path="pytorch_model.bin"):
    
    try:
        upload_file(
            path_or_fileobj = model_path,
            path_in_repo = model_path,
            repo_id = repo_id,
            repo_type = "space"
        )
        print("Model successfully updated on Hugging Face Hub")
    except Exception as e:
        print(f"Error updating model: {e}")

update_model_on_hub(repo_id, "pytorch_model.bin")

  

pytorch_model.bin:   0%|          | 0.00/40.3M [00:00<?, ?B/s]

Model successfully updated on Hugging Face Hub
